<a href="https://colab.research.google.com/github/randywreed/lex_article_review/blob/main/Tuesday_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy git+https://github.com/makcedward/nlpaug.git
!pip install torch>=1.6.0 transformers>=3.0.2
!pip install torch>=1.6.0 fairseq>=0.9.0 sacremoses>=0.0.43 fastBPE>=0.1.0
!pip install nltk>=3.4.5


  Cloning https://github.com/makcedward/nlpaug.git to /tmp/pip-req-build-mmh3uuhx
  Running command git clone -q https://github.com/makcedward/nlpaug.git /tmp/pip-req-build-mmh3uuhx
  Created wheel for nlpaug: filename=nlpaug-1.1.0-cp36-none-any.whl size=823805 sha256=81d9ed5591eede615fe4dd291511efe61f8f8d7955809c36fdb0dc05a302518b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5v3l8ztr/wheels/2b/ef/30/a4e22f9a97373c9ab6763670c94aa5e111b0b956983f3892a4
Successfully built nlpaug


In [2]:
!wget http://nlpgrid.seas.upenn.edu/PPDB/eng/ppdb-2.0-l-all.gz


--2020-12-07 14:03:15--  http://nlpgrid.seas.upenn.edu/PPDB/eng/ppdb-2.0-l-all.gz
Resolving nlpgrid.seas.upenn.edu (nlpgrid.seas.upenn.edu)... 158.130.57.54
Connecting to nlpgrid.seas.upenn.edu (nlpgrid.seas.upenn.edu)|158.130.57.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257367842 (2.1G) [application/x-gzip]
Saving to: ‘ppdb-2.0-l-all.gz’

ppdb-2.0-l-all.gz   100%[===================>]   2.10G  33.7MB/s    in 65s     

2020-12-07 14:04:19 (33.3 MB/s) - ‘ppdb-2.0-l-all.gz’ saved [2257367842/2257367842]



In [3]:
!gunzip ppdb-2.0-l-all.gz

In [4]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [5]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_word2vec(dest_dir='.') # Download word2vec model
DownloadUtil.download_glove(model_name='glove.6B', dest_dir='.') # Download GloVe model
DownloadUtil.download_fasttext(model_name='wiki-news-300d-1M', dest_dir='.') # Download fasttext model

In [6]:
!gunzip -c -S zip GoogleNews-vectors-negative300.zip > GoogleNews-vectors-negative300.bin

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('left_out_creation_main').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
tdf=pd.DataFrame.from_records(rows)
headers=tdf.iloc[0]
df=pd.DataFrame(tdf.values[1:],columns=headers)

In [9]:
import os
os.environ["MODEL_DIR"] = '/content/'
model_dir='/content/'

In [10]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [11]:
df.head()

,,What is left out(%),Creation Content,Creation Content_last,Left_out_flag
0,0,100,In his article titled “Some Real News About Fa...,that affirms their personal opinion. Graham en...,TRUE
1,14,100,News Source (citation): “92% Of Indian Employe...,really portray two sides of the topic discusse...,TRUE
2,20,100,,,TRUE
3,28,100,"The article ""The Story of the Birth of Jesus"" ...",first part of the summary starts off in the Go...,TRUE
4,43,100,The article in the Christian Post called “Half...,not be allowed to pick and choose what they co...,TRUE


In [12]:
#split the paragraph in to sentences.
def split_para(text):
  from nltk import tokenize
  out=tokenize.sent_tokenize(text)
  return out

In [13]:
def keyboard_aug(text):
  aug = nac.KeyboardAug()
  augmented_text = aug.augment(text)
  return augmented_text

In [14]:
def spelling_aug(text):
  aug=naw.SpellingAug()
  augmented_text= aug.augment(text, n=1)
  return augmented_text

In [15]:
def word2vec_aug(text):
  aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='GoogleNews-vectors-negative300.bin',
    action="substitute")
  augmented_text = aug.augment(text)
  return augmented_text

In [16]:
def run_augmentation(func,newaugs,df):
  for idx,row in df.iterrows():
    text=row['Creation Content_last']
    flag=row['Left_out_flag']
    flag=row['']
    sents=split_para(text)
    s=len(sents)
    #print('num of sent {}'.format(s))
    cnt=0
    #change one sentence each time.
    for i in range(s):
      augmented_text=func(sents[i])
      new_text=''
      for k in range(s):
        if k==i:
          new_text+=augmented_text
        else:
          new_text+=sents[k]
      newaugs.append([new_text,flag])
      #print('save augmentation for para changed sent {}'.format(i))
  return newaugs

In [17]:
newaugs=[]
newaugs=run_augmentation(keyboard_aug,newaugs,df)

In [18]:

newaugs=run_augmentation(spelling_aug,newaugs,df)

In [ ]:
newaugs=run_augmentation(word2vec_aug,newaugs,df)

In [19]:
print(len(newaugs))

1572


In [ ]:
'''
newaugs=[]
for idx,row in df.iterrows():
  text=row['Creation Content_last']
  sents=split_para(text)
  s=len(sents)
  #print('num of sent {}'.format(s))
  cnt=0
  #change one sentence each time.
  for i in range(s):
    aug = nac.KeyboardAug()
    #print('changed sentence {}'.format(i))
    augmented_text = aug.augment(sents[i])
    new_text=''
    for k in range(s):
      if k==i:
        new_text+=augmented_text
      else:
        new_text+=sents[k]
    newaugs.append(new_text)
    #print('save augmentation for para changed sent {}'.format(i))
  '''

In [ ]:

%pprint off
newaugs

Pretty printing has been turned ON


["that afeirks their personal opinion.Graham ends his article by discussing the contradictions in the public's understanding and solution of the issue, “made-up news,” stating, “almost two-thirds of people said that political divisions are a big challenge to addressing made-up news.Yet the steps that they report taking themselves seem likely to only exacerbate those political divisions” (Graham).This part of the article reminded me of “The Great Hack,” which points out the role social media and marketing algorithms played in creating a world of simultaneous, yet contradictory truths.Due to these algorithms, fake news and conspiracy theories are fed to those deemed predisposed to their nonsense.The resulting outcome is a world where those identified as holding false opinions have them reinforced by a near-constant stream of false media, which is directed at them by algorithms created to maximize screen time.While Graham’s analysis of the Pew’s study is fairly in depth, I believe there a